In [116]:
using DataFrames
using CSV
using Plots
using Polynomials
using LaTeXStrings

function analytical(x, y)
    return sin(3*pi*x)*sin(4*pi*y)
end

function get_tau(n, L, M, N)
    return 2/((4*(L^2+M^2) + 2*pi^2) + (4*(L^2+M^2)-2*pi^2)*cos(pi*(2*n-1)/(2*N)))
end

function get_n(N, num)
    if N == 1
        push!(num, 1)
        return 1
    end
    
    if N%2 == 0
        push!(num, N)
        get_n(Integer(round(N/2, digits = 0)), num)
    end
    
    if N%2 != 0
        push!(num, N)
        get_n(N-1, num)
    end
end

L = 5
M = 5

hx = 1/(L)
hy = 1/(M)
error0 = 10^(-4)

max = 4*(L^2+M^2)
min = 2*pi^2
N = Integer(cld(log(2/error0),log((sqrt(max)+sqrt(min))/(sqrt(max)-sqrt(min)))))

u = zeros(Float64, (L+1, M+1, N))
num = []
get_n(16, num)
print(num)

num = reverse(num, dims = 1)
thetta = [1]
i = 1
while i <= (length(num) - 2)
    thetta_new = zeros(Float64, num[i+1])
    #print(i)
   # print(thetta_new)
    if num[i+2] == 2*num[i+1]
        for j = 1:num[i]
            thetta_new[2*j-1] = thetta[j]
            thetta_new[2*j] = 4*num[i] - thetta_new[2*j-1]
        end
    end
    
    if (i+2 == length(num)) & (num[i+2]%2 == 0)
        thetta = thetta_new
        thetta_new = zeros(Float64, num[i+2])
        for j = 1:num[i+1]
            thetta_new[2*j - 1] = thetta[j]
            thetta_new[2*j] = 4*num[i+1] - thetta_new[2*j-1] 
        end
    end
    
    
    if num[i+2] == num[i+1] + 1
        for j = 1:num[i]
            thetta_new[2*j-1] = thetta[j]
            thetta_new[2*j] = 4*num[i] - thetta_new[2*j-1] + 2
        end
        push!(thetta_new, num[i+2])
        i = i + 1
    end
    
    i = i + 1
    thetta = thetta_new
    #print(i)
    #print(thetta)
end

print(thetta)

Any[16, 8, 4, 2, 1][1.0, 31.0, 15.0, 17.0, 7.0, 25.0, 9.0, 23.0, 3.0, 29.0, 13.0, 19.0, 5.0, 27.0, 11.0, 21.0]